# The Unitary Events Analysis

The executed version of this tutorial is at https://elephant.readthedocs.io/en/latest/tutorials/unitary_event_analysis.html

The Unitary Events (UE) analysis \[1\] tool allows us to  reliably detect correlated spiking activity that is not explained by the firing rates of the neurons alone. It was designed to detect coordinated spiking activity that occurs significantly more often than predicted by the firing rates of the neurons. The method allows one to analyze correlations not only between pairs of neurons but also between multiple neurons, by considering the various spike patterns across the neurons. In addition, the method allows one to extract the dynamics of correlation between the neurons by perform-ing the analysis in a time-resolved manner. This enables us to relate the occurrence of spike synchrony to behavior.

The algorithm:

1. Align trials, decide on width of analysis window.
2. Decide on allowed coincidence width.
3. Perform a sliding window analysis. In each window:
    1. Detect and count coincidences.
    2. Calculate expected number of coincidences.
    3. Evaluate significance of detected coincidences.
    4. If significant, the window contains Unitary Events.
4. Explore behavioral relevance of UE epochs.

References:

1. Grün, S., Diesmann, M., Grammont, F., Riehle, A., & Aertsen, A. (1999). Detecting unitary events without discretization of time. Journal of neuroscience methods, 94(1), 67-79.

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import quantities as pq
import neo

import elephant.unitary_event_analysis as ue
from viziphant.unitary_event_analysis import plot_ue

Next, we download a data file containing spike train data from multiple trials of two neurons.

In [ ]:
# Download the data
!curl https://web.gin.g-node.org/INM-6/elephant-data/raw/master/dataset-1/dataset-1.h5 --output dataset-1.h5 --location

## Load data and extract spiketrains

In [ ]:
block = neo.io.NeoHdf5IO("./dataset-1.h5").read_block()
sts1 = block.segments[0].spiketrains
sts2 = block.segments[1].spiketrains
spiketrains = np.vstack((sts1,sts2)).T

## Calculate Unitary Events

In [ ]:
Js_dict = ue.jointJ_window_analysis(
    spiketrains, bin_size=5 * pq.ms, winsize=100 * pq.ms, winstep=10 * pq.ms,
    pattern_hash=[3])

And plot the results.

In [ ]:
plot_params = {
    'events': {},
    'unit_real_ids': ['1', '2'],
    'hspace': 0.6,
    'figsize': (12, 10),
    'fsize': 12,
    'ms': 5,
    'lw': 1
}

plot_ue(spiketrains, Js_dict=Js_dict, sig_level=0.05, binsize=5 * pq.ms,
        winsize=100 * pq.ms, winstep=10 * pq.ms,
        pattern_hash=ue.inverse_hash_from_pattern([3], N=2),
        plot_params_user=plot_params);